In [ ]:
from mxnet.gluon import nn
from mxnet import nd

def get_net():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(4,activation="relu"))
        net.add(nn.Dense(2))
    return net

x = nd.random.uniform(shape=(3,5))

不call initialize()

In [ ]:
import sys
try:
    net = get_net()
    net(x)
except RuntimeError as err:
    sys.stderr.write(str(err))

In [ ]:
net.initialize()
net(x)

访问模型参数

In [ ]:
W = net[0].weight
b = net[0].bias
print('name:',net[0].name,'\nweight:',W,'\nbias:',b)

In [ ]:
print('weight',W.data())
print('weight gradient',W.grad())
print('bias',b.data())
print('bias gradient',b.grad())

In [ ]:
params = net.collect_params()
print(params)
print(params['sequential0_dense0_bias'].data())
print(params.get('dense0_weight').data())

In [ ]:
from mxnet import init
params.initialize(init=init.Normal(sigma=0.02),force_reinit=True)
print(net[0].weight.data(),net[0].bias.data())

In [ ]:
params.initialize(init=init.One(),force_reinit=True)
print(net[0].weight.data(),net[0].bias.data())

In [ ]:
class MyInit(init.Initializer):
    def __init__(self):
        super(MyInit,self).__init__()
        self._verbose = True
    def _init_weight(self,_,arr):
        print('init weight',arr.shape)
        nd.random.uniform(low=5,high=10,out=arr)
    def _init_bias(self,_,arr):
        pritn('init bias',arr.shape)
        arr[:] = 2

params.initialize(init=MyInit(),force_reinit=True)
print(net[0].weight.data(),net[0].bias.data())

In [ ]:
net = get_net()
print(net.collect_params())

In [ ]:
net.initialize(init=MyInit())

In [ ]:
net(x)

In [ ]:
print(net.collect_params())

避免延后初始化

In [ ]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(4,in_units=5,activation="relu"))
    net.add(nn.Dense(2,in_units=4))

net.initialize(MyInit())

共享模型参数

In [ ]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(4,in_units=4,activation="relu"))
    net.add(nn.Dense(4,in_units=4,activation="relu",params=net[-1].params))
    net.add(nn.Dense(2,in_units=4))

In [ ]:
net.initialize(MyInit())
print(net[0].weight.data())
print(net[1].weight.data())